In [3]:
!pip install pyarrow

     |████████████████████████████████| 40.8 MB 67 kB/s  eta 0:00:01


In [1]:
import pandas as pd

In [7]:
nyc_taxi_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
nyc_taxi_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

In [8]:
nyc_taxi_jan.shape

(2964624, 19)